In [2]:
import pandas as pd
#import plotly.graph_objects as go
# 출력 제한 해제
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
pd.set_option('display.max_seq_items', None)
pd.options.display.max_colwidth = 100

import warnings
warnings.filterwarnings(action='ignore') 


In [3]:
train = pd.read_csv('train.csv')
test =  pd.read_csv('test.csv')
trade = pd.read_csv('international_trade.csv')

train.head()

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg)
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0


In [4]:
time = pd.to_datetime(train['timestamp'].copy())

train['Date'] = pd.to_datetime(train['timestamp'])
train['week'] = train['Date'].apply(lambda x: x.isocalendar()[1]) # 일요일 제거를 위함
train['day_name'] = train['Date'].dt.day_name()

train['year'] = train['timestamp'].apply(lambda x : int(x[0:4]))
train['month'] = train['timestamp'].apply(lambda x : int(x[5:7]))
train['day'] = train['timestamp'].apply(lambda x : int(x[8:10]))




In [5]:
import datetime
import holidays

# 공휴일 정보를 가져옴
holiday_list = holidays.Korea(years=datetime.datetime.now().year)

# train['holiday'] = 0

# # 특정 날짜가 공휴일인지 확인, 일요일 행은 train할때 다 지운다.
# for i in range(len(train)):
#     date = datetime.date(train.iloc[i,-4], train.iloc[i,-3], train.iloc[i,-2])
#     if date in holiday_list:
#         train.iloc[i,-1] = holiday_list[datetime.date(train.iloc[i, -4], train.iloc[i, -3], train.iloc[i, -2])] #공휴일이면 공휴일 이름 넣음

In [6]:
train.head()

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg),Date,week,day_name,year,month,day
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019-01-01,1,Tuesday,2019,1,1
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019-01-02,1,Wednesday,2019,1,2
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019-01-03,1,Thursday,2019,1,3
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019-01-04,1,Friday,2019,1,4
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019-01-05,1,Saturday,2019,1,5


In [7]:
#시계열 특성을 학습에 반영하기 위해 timestamp를 월, 일, 시간으로 나눕니다
test_re = test.copy()

test_re['year'] = test_re['timestamp'].apply(lambda x : int(x[0:4]))
test_re['month'] = test_re['timestamp'].apply(lambda x : int(x[5:7]))
test_re['day'] = test_re['timestamp'].apply(lambda x : int(x[8:10]))
test_re['Date'] = pd.to_datetime(test_re['timestamp'])
test_re['week'] = test_re['Date'].apply(lambda x: x.isocalendar()[1]) # 일요일 제거를 위함
test_re['day_name'] = test_re['Date'].dt.day_name()


# test_re['holiday'] = 0

# for i in range(len(test_re)):
#     date = datetime.date(test_re.iloc[i,5], test_re.iloc[i,6], test_re.iloc[i,7])
#     if date in holiday_list:
#         test_re.iloc[i,-1] = holiday_list[datetime.date(test_re.iloc[i, 5], test_re.iloc[i, 6], test_re.iloc[i, 7])] #공휴일이면 공휴일 이름 넣음

In [8]:
test_re.head()

,ID,timestamp,item,corporation,location,year,month,day,Date,week,day_name
0,TG_A_J_20230304,2023-03-04,TG,A,J,2023,3,4,2023-03-04,9,Saturday
1,TG_A_J_20230305,2023-03-05,TG,A,J,2023,3,5,2023-03-05,9,Sunday
2,TG_A_J_20230306,2023-03-06,TG,A,J,2023,3,6,2023-03-06,10,Monday
3,TG_A_J_20230307,2023-03-07,TG,A,J,2023,3,7,2023-03-07,10,Tuesday
4,TG_A_J_20230308,2023-03-08,TG,A,J,2023,3,8,2023-03-08,10,Wednesday


In [11]:
from sklearn.preprocessing import LabelEncoder

# 793점 방법
train_x_regression = train[train['day_name'] != 'Sunday'].copy()

train_x_regression = train_x_regression.drop(columns=['ID', 'timestamp', 'supply(kg)','Date'])

train_x_classification = train.drop(columns=['ID', 'timestamp', 'supply(kg)','Date'])

for k in range(len(train_x_classification)):
    if train_x_classification['price(원/kg)'][k] > 0:
        train_x_classification['price(원/kg)'][k] = 1

test_x = test_re.drop(columns=['ID', 'timestamp','Date'])


qual_col = ['item', 'corporation', 'location','day_name']
#del train_x['holiday']

#train_x = pd.get_dummies(train_x, columns=qual_col, prefix=qual_col)
#test_x = pd.get_dummies(test_x, columns=qual_col, prefix=qual_col)

for i in qual_col:
   le = LabelEncoder()
   train_x_classification[i]=le.fit_transform(train_x_classification[i])
   train_x_regression[i]=le.transform(train_x_regression[i]) # reg는 sunday지웠으니..

   test_x[i]=le.transform(test_x[i]) #test 데이터에 대해서 fit하는 것은 data leakage에 해당합니다
    

# %%
#train_x_regression = pd.concat([train_x_regression, train_y], axis=1)
#train_x_classification = pd.concat([train_x_classification, train_y], axis =1)

In [12]:
from pycaret.classification import *

c = setup(train_x_classification, 
          target = 'price(원/kg)',
          fold = 5, 
          #numeric_features=train_x_classification.columns, 
          session_id = 123, 
          normalize = True, 
          normalize_method = 'minmax', 
          train_size=0.8)


,Description,Value
0,Session id,123
1,Target,price(원/kg)
2,Target type,Binary
3,Original data shape,"(59397, 9)"
4,Transformed data shape,"(59397, 9)"
5,Transformed train set shape,"(47517, 9)"
6,Transformed test set shape,"(11880, 9)"
7,Numeric features,8
8,Preprocess,True
9,Imputation type,simple


In [13]:
m = compare_models()

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,20:21:09
Status,. . . . . . . . . . . . . . . . . .,Selecting Estimator
Estimator,. . . . . . . . . . . . . . . . . .,Logistic Regression


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
xgboost,Extreme Gradient Boosting,0.9413,0.9864,0.9462,0.9115,0.9285,0.8787,0.8792,0.5880
catboost,CatBoost Classifier,0.9398,0.9856,0.9483,0.9066,0.9270,0.8758,0.8764,4.8360
lightgbm,Light Gradient Boosting Machine,0.9345,0.9838,0.9494,0.8946,0.9212,0.8652,0.8664,0.4600
rf,Random Forest Classifier,0.9291,0.9792,0.9279,0.8994,0.9134,0.8534,0.8537,0.5220
et,Extra Trees Classifier,0.9270,0.9759,0.9192,0.9016,0.9103,0.8487,0.8489,0.4900
dt,Decision Tree Classifier,0.9129,0.9094,0.8916,0.8922,0.8919,0.8189,0.8189,0.6120
gbc,Gradient Boosting Classifier,0.9050,0.9606,0.8980,0.8704,0.8840,0.8036,0.8039,0.7700
knn,K Neighbors Classifier,0.8850,0.9452,0.8566,0.8579,0.8573,0.7610,0.7610,1.4620
ada,Ada Boost Classifier,0.8507,0.9292,0.7759,0.8416,0.8073,0.6858,0.6874,0.3260
qda,Quadratic Discriminant Analysis,0.7206,0.8104,0.8582,0.6094,0.7125,0.4557,0.4814,0.0360


In [14]:
llar  = create_model('xgboost')
predictions_classfication = predict_model(llar, data = test_x)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9387,0.9863,0.9436,0.9078,0.9254,0.8733,0.8738
1,0.9433,0.9865,0.9488,0.9138,0.9310,0.8829,0.8833
2,0.9410,0.9863,0.9481,0.9094,0.9283,0.8782,0.8787
3,0.9376,0.9857,0.9449,0.9045,0.9243,0.8713,0.8719
4,0.9458,0.9873,0.9457,0.9219,0.9336,0.8878,0.8881
Mean,0.9413,0.9864,0.9462,0.9115,0.9285,0.8787,0.8792
Std,0.0030,0.0005,0.0019,0.0060,0.0035,0.0061,0.0060


In [15]:
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

train_df = pd.read_csv('train.csv')[['ID','timestamp','supply(kg)', 'price(원/kg)']]
train_df['item_id'] = train_df.ID.str[0:6]
data = TimeSeriesDataFrame(train_df.drop(columns=['ID']))
predictor = TimeSeriesPredictor( 
    prediction_length=28,
    target="price(원/kg)",
    eval_metric="RMSE",
)
# seed 고정
predictor.fit( data, random_seed=42, )
predictor.refit_full()


pred = predictor.predict(data, random_seed=42, )


# submission = pd.read_csv('./sample_submission.csv')
# submission['answer'] = pred.reset_index()['mean']
# submission.loc[ submission['answer'] < 0.0, 'answer'] = 0.0
# submission.to_csv('./dacon_submission.csv', index=False)
# submission

================ TimeSeriesPredictor ================
TimeSeriesPredictor.fit() called
Fitting with arguments:
{'enable_ensemble': True,
 'evaluation_metric': 'RMSE',
 'excluded_model_types': None,
 'hyperparameter_tune_kwargs': None,
 'hyperparameters': 'default',
 'num_val_windows': 1,
 'prediction_length': 28,
 'random_seed': 42,
 'target': 'price(원/kg)',
 'time_limit': None,
 'verbosity': 2}
Provided training data set with 59397 rows, 39 items (item = single time series). Average time series length is 1523.0. Data frequency is 'D'.
Global seed set to 42
AutoGluon will save models to AutogluonModels\ag-20231113_112251\
AutoGluon will gauge predictive performance using evaluation metric: 'RMSE'
	This metric's sign has been flipped to adhere to being 'higher is better'. The reported score can be multiplied by -1 to get the metric value.

Provided dataset contains following columns:
	target:           'price(원/kg)'
	past covariates:  ['supply(kg)']

Starting training. Start time is 202

In [16]:
submission = pd.read_csv('./sample_submission.csv')
submission['answer'] = pred.reset_index()['mean']
submission.loc[ submission['answer'] < 0.0, 'answer'] = 0.0
submission.to_csv('./dacon_submission.csv', index=False)
submission

,ID,answer
0,TG_A_J_20230304,3266.210242
1,TG_A_J_20230305,591.939461
2,TG_A_J_20230306,3227.345640
3,TG_A_J_20230307,3482.171226
4,TG_A_J_20230308,3398.489916
...,...,...
1087,RD_F_J_20230327,514.982850
1088,RD_F_J_20230328,505.644167
1089,RD_F_J_20230329,520.969869
1090,RD_F_J_20230330,515.917611


In [18]:
result = pd.DataFrame()
result['classification'] = predictions_classfication['prediction_label']
result['regression'] = submission['answer']
result.head()

,classification,regression
0,1,3266.210242
1,0,591.939461
2,1,3227.345640
3,1,3482.171226
4,1,3398.489916


In [19]:
answer = list()
for i in range(len(result)):
    if result.iloc[i, 0] == 0:
        answer.append(0)
    else:
        answer.append(result.iloc[i,1])
    

In [20]:
for i in range(len(answer)):
    if (i-1) % 7 == 0: # 1, 8, 15, 22
        answer[i] = 0

In [21]:
result1 = pd.DataFrame()
result1['ID'] = test['ID']
result1['answer'] = answer

result1.to_csv('result33_1113.csv', index=False)

In [22]:
result1

,ID,answer
0,TG_A_J_20230304,3266.210242
1,TG_A_J_20230305,0.000000
2,TG_A_J_20230306,3227.345640
3,TG_A_J_20230307,3482.171226
4,TG_A_J_20230308,3398.489916
...,...,...
1087,RD_F_J_20230327,514.982850
1088,RD_F_J_20230328,505.644167
1089,RD_F_J_20230329,520.969869
1090,RD_F_J_20230330,515.917611
